In [72]:
import os
import sys
import torch
%load_ext autoreload
%autoreload 2
sys.path.append('/Users/sarahurbut/aladynoulli2/pyScripts/')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_retrospective_full/enrollment_model_W0.0001_batch_0_10000.pt')

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/1252489767.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clo

In [84]:
%load_ext autoreload
%autoreload 2

%autoreload 2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.spatial.distance import pdist, squareform
from scipy.special import expit
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering  # Add this import
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
pandas2ri.activate()

def load_model_essentials(base_path='/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/'):
    """
    Load all essential components
    """
    print("Loading components...")
    
    # Load large matrices
    Y = torch.load(base_path + 'Y_tensor.pt')
    E = torch.load(base_path + 'E_matrix.pt')
    G = torch.load(base_path + 'G_matrix.pt')
    
    # Load other components
    essentials = torch.load(base_path + 'model_essentials.pt')
    
    print("Loaded all components successfully!")
    
    return Y, E, G, essentials

# Load and initialize model:
Y, E, G, essentials = load_model_essentials()
from clust_huge_amp import *
# Subset the data
Y_100k, E_100k, G_100k, indices = subset_data(Y, E, G, start_index=0, end_index=10000)

torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
# Initialize model with subsetted data

del Y

# Load references (signatures only, no healthy)
refs = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/reference_trajectories.pt')
signature_refs = refs['signature_refs']
# When initializing the model:

readRDS = robjects.r['readRDS']
pce_data = readRDS('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pce_df_prevent.rds')
pce_df = pandas2ri.rpy2py(pce_data)  # Convert to pandas DataFrame
sex=pce_df['Sex'].values

# Convert to numeric: Female=0, Male=1

pce_df['sex_numeric'] = pce_df['Sex'].map({'Female': 0, 'Male': 1}).astype(int)

sex=pce_df['sex_numeric'].values
G_with_sex = ckpt['G']  # sex should be numeric (e.g., 0/1)
# N


model = AladynSurvivalFixedKernelsAvgLoss_clust_logitInit_psitest(
    N=Y_100k.shape[0], 
    D=Y_100k.shape[1], 
    T=Y_100k.shape[2], 
    K=20,
    P=G_with_sex.shape[1],
    init_sd_scaler=1e-1,
    G=G_with_sex, 
    Y=Y_100k,
    genetic_scale=1,
    W=0,
    R=0,
    prevalence_t=essentials['prevalence_t'],
    signature_references=signature_refs,  # Only pass signature refs
    healthy_reference=True,  # Explicitly set to None
    disease_names=essentials['disease_names']
)

torch.manual_seed(0)
np.random.seed(0)
# Initialize with psi and clusters


import cProfile
import pstats
from pstats import SortKey

# Now in your batch run, load and verify:
initial_psi = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/initial_psi_400k.pt')
initial_clusters = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/initial_clusters_400k.pt')

model.initialize_params(true_psi=initial_psi)
model.clusters = initial_clusters
# Verify clusters match
clusters_match = np.array_equal(initial_clusters, model.clusters)
print(f"\nClusters match exactly: {clusters_match}")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Loading components...


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/3330217842.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Y = torch.load(base_path + 'Y_tensor.pt')
/va

Loaded all components successfully!


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/3330217842.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  refs = torch.load('/Users/sarahurbut/Library/


Cluster Sizes:
Cluster 0: 14 diseases
Cluster 1: 7 diseases
Cluster 2: 21 diseases
Cluster 3: 15 diseases
Cluster 4: 17 diseases
Cluster 5: 16 diseases
Cluster 6: 57 diseases
Cluster 7: 18 diseases
Cluster 8: 13 diseases
Cluster 9: 11 diseases
Cluster 10: 18 diseases
Cluster 11: 12 diseases
Cluster 12: 26 diseases
Cluster 13: 7 diseases
Cluster 14: 9 diseases
Cluster 15: 8 diseases
Cluster 16: 7 diseases
Cluster 17: 11 diseases
Cluster 18: 6 diseases
Cluster 19: 55 diseases

Calculating gamma for k=0:
Number of diseases in cluster: 14
Base value (first 5): tensor([-13.8155, -13.8155, -13.1095, -12.4036, -12.4036])
Base value centered (first 5): tensor([-0.3723, -0.3723,  0.3336,  1.0396,  1.0396])
Base value centered mean: 6.57081614008348e-07
Gamma init for k=0 (first 5): tensor([ 0.0026,  0.0050,  0.0103,  0.0132, -0.0116])

Calculating gamma for k=1:
Number of diseases in cluster: 7
Base value (first 5): tensor([-13.8155, -13.8155, -13.8155, -13.8155, -13.8155])
Base value centered

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/3330217842.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  initial_psi = torch.load('/Users/sarahurbut/L


Calculating gamma for k=2:
Number of diseases in cluster: 15.0
Base value (first 5): tensor([-13.8155, -13.8155, -13.1566, -11.8388, -12.4977])
Base value centered (first 5): tensor([-0.3849, -0.3849,  0.2740,  1.5918,  0.9329])
Base value centered mean: 9.290695288655115e-07
Gamma init for k=2 (first 5): tensor([ 0.0016,  0.0073,  0.0100,  0.0140, -0.0118])

Calculating gamma for k=3:
Number of diseases in cluster: 82.0
Base value (first 5): tensor([-13.8155, -13.8155, -13.8155, -13.6950, -13.5744])
Base value centered (first 5): tensor([-0.1026, -0.1026, -0.1026,  0.0179,  0.1384])
Base value centered mean: 4.7445297468584613e-07
Gamma init for k=3 (first 5): tensor([ 0.0012,  0.0003,  0.0016,  0.0018, -0.0006])

Calculating gamma for k=4:
Number of diseases in cluster: 5.0
Base value (first 5): tensor([-13.8155,  -9.8620, -13.8155, -13.8155, -13.8155])
Base value centered (first 5): tensor([-0.1133,  3.8402, -0.1133, -0.1133, -0.1133])
Base value centered mean: -2.841758714566822e-

# Gold standard

In [75]:
from fig5utils import *
disease_names=essentials['disease_names']
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_retrospective_full/enrollment_model_W0.0001_batch_0_10000.pt')
model.load_state_dict(ckpt['model_state_dict'])

evaluate_major_diseases_wsex_with_bootstrap_dynamic(model, Y_100k, E_100k, disease_names, pce_df, n_bootstraps=100, follow_up_duration_years=30, patient_indices=None)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/419753689.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clou


Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.863 (0.827-0.897) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 136 (5.4%) (from 2541 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.711 (0.669-0.752) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 137 (5.4%) (from 2541 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.708 (0.589-0.813) (calculated on 2532 individuals)
Events (10-Year in Eval Cohort): 29 (1.1%) (from 2532 individuals)
Excluded 9 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.569 (0.462-0.667) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 33 (1.3%) (from 2541 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.668 (0.608-0.732) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 94 (3.7%) (from 2

{'ASCVD': {'auc': 0.8629968203497616,
  'n_events': 136,
  'event_rate': 5.352223534041716,
  'ci_lower': 0.82711792821039,
  'ci_upper': 0.8972171076423406},
 'Diabetes': {'auc': 0.7110503175971921,
  'n_events': 137,
  'event_rate': 5.3915781188508465,
  'ci_lower': 0.6686920009741962,
  'ci_upper': 0.752473153708459},
 'Atrial_Fib': {'auc': 0.7078815765908496,
  'n_events': 29,
  'event_rate': 1.1453396524486572,
  'ci_lower': 0.5888396699054479,
  'ci_upper': 0.8130167874025415},
 'CKD': {'auc': 0.5687496979363008,
  'n_events': 33,
  'event_rate': 1.2987012987012987,
  'ci_lower': 0.46240219454795806,
  'ci_upper': 0.6672354811645976},
 'All_Cancers': {'auc': 0.6684998565329671,
  'n_events': 94,
  'event_rate': 3.699330972058245,
  'ci_lower': 0.6075515125374636,
  'ci_upper': 0.7324619478085796},
 'Stroke': {'auc': 0.5155847603876278,
  'n_events': 30,
  'event_rate': 1.1806375442739079,
  'ci_lower': 0.41902427383886715,
  'ci_upper': 0.629165494061507},
 'Heart_Failure': {'auc

# fixed phi from enrollment

In [91]:

from fig5utils import *
disease_names=essentials['disease_names']
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_predictions_fixedphi_withpcs/output/model_enroll_fixedphi_sex_0_10000.pt')
model.load_state_dict(ckpt['model_state_dict'])

evaluate_major_diseases_wsex_with_bootstrap_dynamic(model, Y_100k, E_100k, disease_names, pce_df, n_bootstraps=100, follow_up_duration_years=30, patient_indices=None)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/1696711840.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clo


Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.579 (0.517-0.625) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 136 (5.4%) (from 2541 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.547 (0.502-0.591) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 137 (5.4%) (from 2541 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.531 (0.445-0.627) (calculated on 2532 individuals)
Events (10-Year in Eval Cohort): 29 (1.1%) (from 2532 individuals)
Excluded 9 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.585 (0.492-0.694) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 33 (1.3%) (from 2541 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.575 (0.514-0.621) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 94 (3.7%) (from 2

{'ASCVD': {'auc': 0.5793873058578941,
  'n_events': 136,
  'event_rate': 5.352223534041716,
  'ci_lower': 0.5166386911472844,
  'ci_upper': 0.6248502541121312},
 'Diabetes': {'auc': 0.5468410313710725,
  'n_events': 137,
  'event_rate': 5.3915781188508465,
  'ci_lower': 0.5017127233711027,
  'ci_upper': 0.5905136959868277},
 'Atrial_Fib': {'auc': 0.5310317274443082,
  'n_events': 29,
  'event_rate': 1.1453396524486572,
  'ci_lower': 0.4448442792993816,
  'ci_upper': 0.6273670890723894},
 'CKD': {'auc': 0.5852907061041033,
  'n_events': 33,
  'event_rate': 1.2987012987012987,
  'ci_lower': 0.49246689710073993,
  'ci_upper': 0.6936970406569458},
 'All_Cancers': {'auc': 0.5753506247337166,
  'n_events': 94,
  'event_rate': 3.699330972058245,
  'ci_lower': 0.514313517156167,
  'ci_upper': 0.6212059367174722},
 'Stroke': {'auc': 0.5163414310367715,
  'n_events': 30,
  'event_rate': 1.1806375442739079,
  'ci_lower': 0.3910944088711865,
  'ci_upper': 0.6155426367230397},
 'Heart_Failure': {'a

In [79]:
from utils import *
results = evaluate_major_diseases_wsex_with_bootstrap(
    model=model,
    Y_100k=Y_100k,
    E_100k=E_100k,
    disease_names=disease_names,
    pce_df=pce_df,
    n_bootstraps=20,
    follow_up_duration_years=10,
)


Evaluating ASCVD (10-Year Outcome, 1-Year Score)...
AUC: 0.683 (0.667-0.692) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 831 (8.3%) (from 10000 individuals)
Excluded 0 prevalent cases for ASCVD.

   Sex-stratified analysis:
   Female: AUC = 0.669, Events = 273/5409
   Male: AUC = 0.685, Events = 558/4591

   ASCVD risk in patients with pre-existing conditions:
   RA: AUC = 0.707, Events = 8/37
   Breast_Cancer: AUC = 0.549, Events = 13/170

Evaluating Diabetes (10-Year Outcome, 1-Year Score)...
AUC: 0.613 (0.583-0.631) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 581 (5.8%) (from 10000 individuals)
Excluded 0 prevalent cases for Diabetes.

   Sex-stratified analysis:
   Female: AUC = 0.604, Events = 264/5409
   Male: AUC = 0.617, Events = 317/4591

Evaluating Atrial_Fib (10-Year Outcome, 1-Year Score)...
AUC: 0.701 (0.671-0.719) (calculated on 9864 individuals)
Events (10-Year in Eval Cohort): 376 (3.8%) (from 10000 individuals)
Excluded 13

# joint 10 year

In [86]:

from fig5utils import *
disease_names=essentials['disease_names']
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_prediction_jointphi_sex_pcs/enrollment_model_W0.0001_batch_0_10000.pt')
model.load_state_dict(ckpt['model_state_dict'])

evaluate_major_diseases_wsex_with_bootstrap_dynamic(model, Y_100k, E_100k, disease_names, pce_df, n_bootstraps=100, follow_up_duration_years=20, patient_indices=None)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/4141852495.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clo


Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.717 (0.697-0.738) (calculated on 6424 individuals)
Events (10-Year in Eval Cohort): 578 (9.0%) (from 6424 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.641 (0.617-0.665) (calculated on 6424 individuals)
Events (10-Year in Eval Cohort): 464 (7.2%) (from 6424 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.577 (0.532-0.618) (calculated on 6371 individuals)
Events (10-Year in Eval Cohort): 137 (2.2%) (from 6371 individuals)
Excluded 53 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.654 (0.620-0.693) (calculated on 6424 individuals)
Events (10-Year in Eval Cohort): 173 (2.7%) (from 6424 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.647 (0.625-0.677) (calculated on 6424 individuals)
Events (10-Year in Eval Cohort): 396 (6.2%) (fr

{'ASCVD': {'auc': 0.7168187634877661,
  'n_events': 578,
  'event_rate': 8.997509339975093,
  'ci_lower': 0.6971406970520175,
  'ci_upper': 0.7383857218331997},
 'Diabetes': {'auc': 0.6413323015505671,
  'n_events': 464,
  'event_rate': 7.22291407222914,
  'ci_lower': 0.6171004831653426,
  'ci_upper': 0.6653732160826513},
 'Atrial_Fib': {'auc': 0.5767114177257282,
  'n_events': 137,
  'event_rate': 2.1503688588918535,
  'ci_lower': 0.531748768462853,
  'ci_upper': 0.6182185069180305},
 'CKD': {'auc': 0.6536924034351035,
  'n_events': 173,
  'event_rate': 2.6930261519302614,
  'ci_lower': 0.6199344402579027,
  'ci_upper': 0.6934320493373844},
 'All_Cancers': {'auc': 0.6465882279999731,
  'n_events': 396,
  'event_rate': 6.164383561643835,
  'ci_lower': 0.6249466566810339,
  'ci_upper': 0.6765189587212519},
 'Stroke': {'auc': 0.6098654529218508,
  'n_events': 119,
  'event_rate': 1.852428393524284,
  'ci_lower': 0.5586284446405901,
  'ci_upper': 0.6608779495477444},
 'Heart_Failure': {'a

In [83]:
results = evaluate_major_diseases_wsex_with_bootstrap(
    model=model,
    Y_100k=Y_100k,
    E_100k=E_100k,
    disease_names=disease_names,
    pce_df=pce_df,
    n_bootstraps=20,
    follow_up_duration_years=10,
)


Evaluating ASCVD (10-Year Outcome, 1-Year Score)...
AUC: 0.712 (0.701-0.730) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 831 (8.3%) (from 10000 individuals)
Excluded 0 prevalent cases for ASCVD.

   Sex-stratified analysis:
   Female: AUC = 0.666, Events = 273/5409
   Male: AUC = 0.702, Events = 558/4591

   ASCVD risk in patients with pre-existing conditions:
   RA: AUC = 0.677, Events = 8/37
   Breast_Cancer: AUC = 0.631, Events = 13/170

Evaluating Diabetes (10-Year Outcome, 1-Year Score)...
AUC: 0.628 (0.606-0.642) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 581 (5.8%) (from 10000 individuals)
Excluded 0 prevalent cases for Diabetes.

   Sex-stratified analysis:
   Female: AUC = 0.618, Events = 264/5409
   Male: AUC = 0.628, Events = 317/4591

Evaluating Atrial_Fib (10-Year Outcome, 1-Year Score)...
AUC: 0.684 (0.665-0.707) (calculated on 9864 individuals)
Events (10-Year in Eval Cohort): 376 (3.8%) (from 10000 individuals)
Excluded 13

In [31]:
from fig5utils import *
disease_names=essentials['disease_names']
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_prediction_jointphi_sex_pcs/enrollment_model_W0.0001_batch_0_10000.pt')
model.load_state_dict(ckpt['model_state_dict'])

evaluate_major_diseases_wsex_with_bootstrap_dynamic(model, Y_100k, E_100k, disease_names, pce_df, n_bootstraps=100, follow_up_duration_years=30, patient_indices=None)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/1198762482.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clo


Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.757 (0.720-0.800) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 136 (5.4%) (from 2541 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.658 (0.612-0.691) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 137 (5.4%) (from 2541 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.611 (0.519-0.714) (calculated on 2532 individuals)
Events (10-Year in Eval Cohort): 29 (1.1%) (from 2532 individuals)
Excluded 9 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.579 (0.475-0.684) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 33 (1.3%) (from 2541 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.605 (0.551-0.655) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 94 (3.7%) (from 2

{'ASCVD': {'auc': 0.7571909013085484,
  'n_events': 136,
  'event_rate': 5.352223534041716,
  'ci_lower': 0.7196164322704525,
  'ci_upper': 0.7998098617765688},
 'Diabetes': {'auc': 0.6577358903044803,
  'n_events': 137,
  'event_rate': 5.3915781188508465,
  'ci_lower': 0.6123458384182642,
  'ci_upper': 0.6913152968579725},
 'Atrial_Fib': {'auc': 0.6105087687877995,
  'n_events': 29,
  'event_rate': 1.1453396524486572,
  'ci_lower': 0.5187187812187812,
  'ci_upper': 0.7139761218304131},
 'CKD': {'auc': 0.5791406891885361,
  'n_events': 33,
  'event_rate': 1.2987012987012987,
  'ci_lower': 0.4751873972532305,
  'ci_upper': 0.6840220317703328},
 'All_Cancers': {'auc': 0.6049178759923136,
  'n_events': 94,
  'event_rate': 3.699330972058245,
  'ci_lower': 0.5511648593714323,
  'ci_upper': 0.6551751341489507},
 'Stroke': {'auc': 0.5132218239745122,
  'n_events': 30,
  'event_rate': 1.1806375442739079,
  'ci_lower': 0.42933241580915205,
  'ci_upper': 0.601101468305172},
 'Heart_Failure': {'a

## check joint (we know this was run with enrollment)

In [68]:
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/DB_backup_5132025941p/enrollment_model_W0.0001_jointphi_sexspecific.pt')
ckpt['G'].shape


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/4135310562.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clo

torch.Size([10000, 37])

In [69]:

G_with_sex = ckpt['G']  # sex should be numeric (e.g., 0/1)
# N


model = AladynSurvivalFixedKernelsAvgLoss_clust_logitInit_psitest(
    N=Y_100k.shape[0], 
    D=Y_100k.shape[1], 
    T=Y_100k.shape[2], 
    K=20,
    P=G_with_sex.shape[1],
    init_sd_scaler=1e-1,
    G=G_with_sex, 
    Y=Y_100k,
    genetic_scale=1,
    W=0,
    R=0,
    prevalence_t=essentials['prevalence_t'],
    signature_references=signature_refs,  # Only pass signature refs
    healthy_reference=True,  # Explicitly set to None
    disease_names=essentials['disease_names']
)

model.load_state_dict(ckpt['model_state_dict'])

evaluate_major_diseases_wsex_with_bootstrap_dynamic(model, Y_100k, E_100k, disease_names, pce_df, n_bootstraps=100, follow_up_duration_years=30, patient_indices=None)

/Users/sarahurbut/aladynoulli2/pyScripts/clust_huge_amp.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.signature_refs = torch.tensor(signature_references, dtype=torch.float32)
/Users/sarahurbut/aladynoulli2/pyScripts/clust_huge_amp.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.G = torch.tensor(G, dtype=torch.float32)
/Users/sarahurbut/aladynoulli2/pyScripts/clust_huge_amp.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.G = torch.tensor(G_scaled, dtype=torch.float32)
/Users/sarahurbut/aladynoulli2/pyScripts


Cluster Sizes:
Cluster 0: 14 diseases
Cluster 1: 7 diseases
Cluster 2: 21 diseases
Cluster 3: 15 diseases
Cluster 4: 17 diseases
Cluster 5: 16 diseases
Cluster 6: 57 diseases
Cluster 7: 18 diseases
Cluster 8: 13 diseases
Cluster 9: 11 diseases
Cluster 10: 18 diseases
Cluster 11: 12 diseases
Cluster 12: 26 diseases
Cluster 13: 7 diseases
Cluster 14: 9 diseases
Cluster 15: 8 diseases
Cluster 16: 7 diseases
Cluster 17: 11 diseases
Cluster 18: 6 diseases
Cluster 19: 55 diseases

Calculating gamma for k=0:
Number of diseases in cluster: 14
Base value (first 5): tensor([-13.8155, -13.8155, -13.1095, -12.4036, -12.4036])
Base value centered (first 5): tensor([-0.3723, -0.3723,  0.3336,  1.0396,  1.0396])
Base value centered mean: 6.57081614008348e-07
Gamma init for k=0 (first 5): tensor([ 0.0008,  0.0071,  0.0117,  0.0152, -0.0106])

Calculating gamma for k=1:
Number of diseases in cluster: 7
Base value (first 5): tensor([-13.8155, -13.8155, -13.8155, -13.8155, -13.8155])
Base value centered

{'ASCVD': {'auc': 0.7532988871224164,
  'n_events': 136,
  'event_rate': 5.352223534041716,
  'ci_lower': 0.7116539386338724,
  'ci_upper': 0.8004184426747034},
 'Diabetes': {'auc': 0.6219955791442486,
  'n_events': 137,
  'event_rate': 5.3915781188508465,
  'ci_lower': 0.5644707291331775,
  'ci_upper': 0.6705135730675604},
 'Atrial_Fib': {'auc': 0.6089520162012482,
  'n_events': 29,
  'event_rate': 1.1453396524486572,
  'ci_lower': 0.5070816213147761,
  'ci_upper': 0.7141575415994241},
 'CKD': {'auc': 0.5802764486975013,
  'n_events': 33,
  'event_rate': 1.2987012987012987,
  'ci_lower': 0.4942235278556403,
  'ci_upper': 0.6936778422873241},
 'All_Cancers': {'auc': 0.6035005956055614,
  'n_events': 94,
  'event_rate': 3.699330972058245,
  'ci_lower': 0.552498107718909,
  'ci_upper': 0.6638416804572402},
 'Stroke': {'auc': 0.5130227001194743,
  'n_events': 30,
  'event_rate': 1.1806375442739079,
  'ci_lower': 0.40827787487122774,
  'ci_upper': 0.6241064309256734},
 'Heart_Failure': {'a

## joint fun

In [65]:
from fig5utils import *
pce_df_full=pd.read_csv('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pce_prevent_full.csv')
pce_df_subset=pce_df_full[0:10000]
disease_names=essentials['disease_names']
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_prediction_jointphi_sex_pcs/enrollment_model_W0.0001_batch_0_10000.pt')
model.load_state_dict(ckpt['model_state_dict'])

evaluate_major_diseases_wsex_with_bootstrap_dynamic(model, ckpt['Y'], E_100k, disease_names, pce_df_subset, n_bootstraps=100, follow_up_duration_years=30, patient_indices=None)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/2486174665.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clo


Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.757 (0.716-0.800) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 136 (5.4%) (from 2541 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.658 (0.616-0.695) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 137 (5.4%) (from 2541 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.611 (0.517-0.721) (calculated on 2532 individuals)
Events (10-Year in Eval Cohort): 29 (1.1%) (from 2532 individuals)
Excluded 9 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.579 (0.475-0.688) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 33 (1.3%) (from 2541 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.605 (0.540-0.658) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 94 (3.7%) (from 2

{'ASCVD': {'auc': 0.7571909013085484,
  'n_events': 136,
  'event_rate': 5.352223534041716,
  'ci_lower': 0.7162736593401398,
  'ci_upper': 0.7996759094788214},
 'Diabetes': {'auc': 0.6577358903044803,
  'n_events': 137,
  'event_rate': 5.3915781188508465,
  'ci_lower': 0.6157588508726881,
  'ci_upper': 0.6953519024959631},
 'Atrial_Fib': {'auc': 0.6105087687877995,
  'n_events': 29,
  'event_rate': 1.1453396524486572,
  'ci_lower': 0.5168071313827604,
  'ci_upper': 0.7208575656673831},
 'CKD': {'auc': 0.5791406891885361,
  'n_events': 33,
  'event_rate': 1.2987012987012987,
  'ci_lower': 0.47517277686016224,
  'ci_upper': 0.6882456490166603},
 'All_Cancers': {'auc': 0.6049178759923136,
  'n_events': 94,
  'event_rate': 3.699330972058245,
  'ci_lower': 0.5401905412145855,
  'ci_upper': 0.6578950343016632},
 'Stroke': {'auc': 0.5132218239745122,
  'n_events': 30,
  'event_rate': 1.1806375442739079,
  'ci_lower': 0.4113602215301302,
  'ci_upper': 0.6065410183110185},
 'Heart_Failure': {'

In [ ]:
from fig5utils import *
pce_df_full=pd.read_csv('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pce_prevent_full.csv')
pce_df_subset=pce_df_full[10000:20000]
disease_names=essentials['disease_names']
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_prediction_jointphi_sex_pcs/enrollment_model_W0.0001_batch_10000_20000.pt')
model.load_state_dict(ckpt['model_state_dict'])

evaluate_major_diseases_wsex_with_bootstrap_dynamic(model, ckpt['Y'], E_100k, disease_names, pce_df_subset, n_bootstraps=100, follow_up_duration_years=30, patient_indices=None)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/1476358432.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clo


Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.679 (0.640-0.725) (calculated on 2503 individuals)
Events (10-Year in Eval Cohort): 143 (5.7%) (from 2503 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.708 (0.656-0.750) (calculated on 2503 individuals)
Events (10-Year in Eval Cohort): 118 (4.7%) (from 2503 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.750 (0.678-0.833) (calculated on 2492 individuals)
Events (10-Year in Eval Cohort): 30 (1.2%) (from 2492 individuals)
Excluded 11 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.533 (0.432-0.585) (calculated on 2503 individuals)
Events (10-Year in Eval Cohort): 27 (1.1%) (from 2503 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.616 (0.563-0.669) (calculated on 2503 individuals)
Events (10-Year in Eval Cohort): 77 (3.1%) (from 

{'ASCVD': {'auc': 0.6791958041958043,
  'n_events': 143,
  'event_rate': 5.713144226927687,
  'ci_lower': 0.6396737797737561,
  'ci_upper': 0.7249290061207281},
 'Diabetes': {'auc': 0.707799452794656,
  'n_events': 118,
  'event_rate': 4.714342788653616,
  'ci_lower': 0.6564537185995166,
  'ci_upper': 0.7495627656114395},
 'Atrial_Fib': {'auc': 0.7498375304630382,
  'n_events': 30,
  'event_rate': 1.2038523274478332,
  'ci_lower': 0.6780413780817799,
  'ci_upper': 0.8332437048990318},
 'CKD': {'auc': 0.5333423083826961,
  'n_events': 27,
  'event_rate': 1.0787055533359968,
  'ci_lower': 0.4323761015044704,
  'ci_upper': 0.5851480341504525},
 'All_Cancers': {'auc': 0.6156786329910815,
  'n_events': 77,
  'event_rate': 3.076308429884139,
  'ci_lower': 0.5626293450837618,
  'ci_upper': 0.6686342668863261},
 'Stroke': {'auc': 0.5544491769310465,
  'n_events': 21,
  'event_rate': 0.8389932081502196,
  'ci_lower': 0.43437827617937297,
  'ci_upper': 0.6897797324307383},
 'Heart_Failure': {'au

In [63]:
from fig5utils import *
pce_df_full=pd.read_csv('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pce_prevent_full.csv')
pce_df_subset=pce_df_full[20000:30000]
disease_names=essentials['disease_names']
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_prediction_jointphi_sex_pcs/enrollment_model_W0.0001_batch_20000_30000.pt')
model.load_state_dict(ckpt['model_state_dict'])

evaluate_major_diseases_wsex_with_bootstrap_dynamic(model, ckpt['Y'], E_100k, disease_names, pce_df_subset, n_bootstraps=100, follow_up_duration_years=30, patient_indices=None)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/1205554614.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clo


Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.703 (0.664-0.746) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 143 (5.6%) (from 2541 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.696 (0.652-0.734) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 167 (6.6%) (from 2541 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.660 (0.573-0.758) (calculated on 2536 individuals)
Events (10-Year in Eval Cohort): 30 (1.2%) (from 2536 individuals)
Excluded 5 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.506 (0.433-0.590) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 42 (1.7%) (from 2541 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.606 (0.541-0.665) (calculated on 2541 individuals)
Events (10-Year in Eval Cohort): 80 (3.1%) (from 2

{'ASCVD': {'auc': 0.702703884939081,
  'n_events': 143,
  'event_rate': 5.627705627705628,
  'ci_lower': 0.6639360357402131,
  'ci_upper': 0.7457842727039136},
 'Diabetes': {'auc': 0.696328488768041,
  'n_events': 167,
  'event_rate': 6.572215663124754,
  'ci_lower': 0.6521787933323038,
  'ci_upper': 0.7340838427096437},
 'Atrial_Fib': {'auc': 0.6600425645118382,
  'n_events': 30,
  'event_rate': 1.1829652996845426,
  'ci_lower': 0.5733488476927109,
  'ci_upper': 0.7577966330387702},
 'CKD': {'auc': 0.5060881495455325,
  'n_events': 42,
  'event_rate': 1.6528925619834711,
  'ci_lower': 0.43313142691818596,
  'ci_upper': 0.5897366248250617},
 'All_Cancers': {'auc': 0.6062677773262901,
  'n_events': 80,
  'event_rate': 3.1483667847304213,
  'ci_lower': 0.5414213750606394,
  'ci_upper': 0.6651409677305284},
 'Stroke': {'auc': 0.6214721063035361,
  'n_events': 29,
  'event_rate': 1.1412829594647778,
  'ci_lower': 0.5452880118318795,
  'ci_upper': 0.7040282353836513},
 'Heart_Failure': {'au

In [64]:
from fig5utils import *
pce_df_subset=pce_df_full[330000:340000]
disease_names=essentials['disease_names']
ckpt=torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_prediction_jointphi_sex_pcs/enrollment_model_W0.0001_batch_330000_340000.pt')
model.load_state_dict(ckpt['model_state_dict'])

evaluate_major_diseases_wsex_with_bootstrap_dynamic(model, ckpt['Y'], E_100k, disease_names, pce_df_subset, n_bootstraps=100, follow_up_duration_years=30, patient_indices=None)

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/1180716549.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt=torch.load('/Users/sarahurbut/Library/Clo


Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.686 (0.641-0.733) (calculated on 2562 individuals)
Events (10-Year in Eval Cohort): 151 (5.9%) (from 2562 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.695 (0.647-0.746) (calculated on 2562 individuals)
Events (10-Year in Eval Cohort): 140 (5.5%) (from 2562 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.458 (0.377-0.550) (calculated on 2554 individuals)
Events (10-Year in Eval Cohort): 26 (1.0%) (from 2554 individuals)
Excluded 8 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.619 (0.531-0.720) (calculated on 2562 individuals)
Events (10-Year in Eval Cohort): 31 (1.2%) (from 2562 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.568 (0.512-0.635) (calculated on 2562 individuals)
Events (10-Year in Eval Cohort): 80 (3.1%) (from 2

{'ASCVD': {'auc': 0.6864536437575022,
  'n_events': 151,
  'event_rate': 5.893832943013271,
  'ci_lower': 0.6408218399773111,
  'ci_upper': 0.7333201768648648},
 'Diabetes': {'auc': 0.6948478235224727,
  'n_events': 140,
  'event_rate': 5.46448087431694,
  'ci_lower': 0.6473089499538704,
  'ci_upper': 0.7462902778905613},
 'Atrial_Fib': {'auc': 0.45752190847127555,
  'n_events': 26,
  'event_rate': 1.0180109631949883,
  'ci_lower': 0.3765504663397537,
  'ci_upper': 0.549752759702033},
 'CKD': {'auc': 0.6191483667044775,
  'n_events': 31,
  'event_rate': 1.209992193598751,
  'ci_lower': 0.5314332651789754,
  'ci_upper': 0.7200861253930616},
 'All_Cancers': {'auc': 0.5682514101531023,
  'n_events': 80,
  'event_rate': 3.12256049960968,
  'ci_lower': 0.5120910812711724,
  'ci_upper': 0.6353951505902593},
 'Stroke': {'auc': 0.5228362227475702,
  'n_events': 24,
  'event_rate': 0.936768149882904,
  'ci_lower': 0.4250125110311483,
  'ci_upper': 0.6421851140802983},
 'Heart_Failure': {'auc': 

In [ ]:
from fig5utils import *
import pandas as pd
import numpy as np
import torch

# Load full pce_df
pce_df_full = pd.read_csv('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/pce_prevent_full.csv')
disease_names = essentials['disease_names']

# Storage for results - Dynamic predictions
joint_10yr_results = []
joint_30yr_results = []
fixed_10yr_results = []
fixed_30yr_results = []

# Storage for results - Static predictions (1-year score for 10-year outcome)
joint_static_10yr_results = []
fixed_static_10yr_results = []

# Loop through checkpoints 0-40 (batch_0_10000 to batch_390000_400000)
for batch_idx in range(41):
    start_idx = batch_idx * 10000
    end_idx = (batch_idx + 1) * 10000
    
    print(f"\n{'='*80}")
    print(f"Processing batch {batch_idx}: {start_idx} to {end_idx}")
    print(f"{'='*80}")
    
    # Get pce_df subset for this batch
    pce_df_subset = pce_df_full[start_idx:end_idx].copy().reset_index(drop=True)
    
    # === JOINT PHI CHECKPOINTS ===
    joint_ckpt_path = f'/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_prediction_jointphi_sex_pcs/enrollment_model_W0.0001_batch_{start_idx}_{end_idx}.pt'
    
    try:
        joint_ckpt = torch.load(joint_ckpt_path, weights_only=False)
        model.load_state_dict(joint_ckpt['model_state_dict'])
        
        # Use Y from checkpoint and update model.Y so forward() uses correct patients
        Y_batch = joint_ckpt['Y']
        model.Y = torch.tensor(Y_batch, dtype=torch.float32)
        model.N = Y_batch.shape[0]  # Update N to match new Y size
        
        # 10-year predictions
        print(f"\nJoint Phi - 10 year predictions...")
        joint_10yr = evaluate_major_diseases_wsex_with_bootstrap_dynamic(
            model, Y_batch, E_100k, disease_names, pce_df_subset, 
            n_bootstraps=100, follow_up_duration_years=10, patient_indices=None
        )
        joint_10yr['batch_idx'] = batch_idx
        joint_10yr_results.append(joint_10yr)
        
        # 30-year predictions
        print(f"\nJoint Phi - 30 year predictions...")
        joint_30yr = evaluate_major_diseases_wsex_with_bootstrap_dynamic(
            model, Y_batch, E_100k, disease_names, pce_df_subset, 
            n_bootstraps=100, follow_up_duration_years=30, patient_indices=None
        )
        joint_30yr['batch_idx'] = batch_idx
        joint_30yr_results.append(joint_30yr)
        
        # Static 10-year predictions (using 1-year score)
        print(f"\nJoint Phi - Static 10 year predictions (1-year score)...")
        joint_static_10yr = evaluate_major_diseases_wsex_with_bootstrap(
            model=model,
            Y_100k=Y_batch,
            E_100k=E_100k,
            disease_names=disease_names,
            pce_df=pce_df_subset,
            n_bootstraps=100,
            follow_up_duration_years=10,
        )
        joint_static_10yr['batch_idx'] = batch_idx
        joint_static_10yr_results.append(joint_static_10yr)
        
    except FileNotFoundError:
        print(f"Joint phi checkpoint not found: {joint_ckpt_path}")
        continue
    except Exception as e:
        print(f"Error processing joint phi checkpoint {batch_idx}: {e}")
        continue
    
    # === FIXED PHI CHECKPOINTS ===
    fixed_ckpt_path = f'/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_predictions_fixedphi_withpcs_fromclaudeoutput/model_enroll_fixedphi_sex_{start_idx}_{end_idx}.pt'
    
    try:
        fixed_ckpt = torch.load(fixed_ckpt_path, weights_only=False)
        model.load_state_dict(fixed_ckpt['model_state_dict'])
        
        # For fixed phi, Y is not saved in checkpoint, so extract from full Y tensor
        # Load full Y tensor if not already available
        if 'Y_full' not in globals():
            Y_full = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/Y_tensor.pt')
        if 'E_full' not in globals():
            E_full = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/E_enrollment_full.pt')
        
        # Extract batch from full tensors
        Y_batch = Y_full[start_idx:end_idx]
        E_batch = E_full[start_idx:end_idx]
        
        # Update model.Y and model.N so forward() uses correct patients
        model.Y = torch.tensor(Y_batch, dtype=torch.float32)
        model.N = Y_batch.shape[0]  # Update N to match new Y size
       
        # 10-year predictions
        print(f"\nFixed Phi - 10 year predictions...")
        fixed_10yr = evaluate_major_diseases_wsex_with_bootstrap_dynamic(
            model, Y_batch, E_batch, disease_names, pce_df_subset, 
            n_bootstraps=100, follow_up_duration_years=10, patient_indices=None
        )
        fixed_10yr['batch_idx'] = batch_idx
        fixed_10yr_results.append(fixed_10yr)
        
        # 30-year predictions
        print(f"\nFixed Phi - 30 year predictions...")
        fixed_30yr = evaluate_major_diseases_wsex_with_bootstrap_dynamic(
            model, Y_batch, E_batch, disease_names, pce_df_subset, 
            n_bootstraps=100, follow_up_duration_years=30, patient_indices=None
        )
        fixed_30yr['batch_idx'] = batch_idx
        fixed_30yr_results.append(fixed_30yr)
        
        # Static 10-year predictions (using 1-year score)
        print(f"\nFixed Phi - Static 10 year predictions (1-year score)...")
        fixed_static_10yr = evaluate_major_diseases_wsex_with_bootstrap(
            model=model,
            Y_100k=Y_batch,
            E_100k=E_batch,
            disease_names=disease_names,
            pce_df=pce_df_subset,
            n_bootstraps=100,
            follow_up_duration_years=10,
        )
        fixed_static_10yr['batch_idx'] = batch_idx
        fixed_static_10yr_results.append(fixed_static_10yr)
        
    except FileNotFoundError:
        print(f"Fixed phi checkpoint not found: {fixed_ckpt_path}")
        continue
    except Exception as e:
        print(f"Error processing fixed phi checkpoint {batch_idx}: {e}")
        continue

print(f"\n{'='*80}")
print("Completed processing all checkpoints!")
print(f"{'='*80}")



Processing batch 0: 0 to 10000

Joint Phi - 10 year predictions...

Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.655 (0.637-0.674) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 831 (8.3%) (from 10000 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.578 (0.558-0.606) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 581 (5.8%) (from 10000 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.525 (0.504-0.545) (calculated on 9864 individuals)
Events (10-Year in Eval Cohort): 376 (3.8%) (from 9864 individuals)
Excluded 136 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.710 (0.679-0.738) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 207 (2.1%) (from 10000 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.660 (0.636-0.681) (calc

In [ ]:
# Rerun only Fixed Phi evaluations with proper Y batch extraction
# (Joint phi results should already be in joint_*_results variables)

# Clear only fixed phi results to rerun them
fixed_10yr_results = []
fixed_30yr_results = []
fixed_static_10yr_results = []

# Load full Y and E tensors once
print("Loading full Y and E tensors...")
Y_full = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/Y_tensor.pt')
E_full = torch.load('/Users/sarahurbut/Library/CloudStorage/Dropbox-Personal/data_for_running/E_enrollment_full.pt')
print("Loaded full tensors!")

# Loop through checkpoints 0-40 (batch_0_10000 to batch_390000_400000)
for batch_idx in range(41):
    start_idx = batch_idx * 10000
    end_idx = (batch_idx + 1) * 10000
    
    print(f"\n{'='*80}")
    print(f"Processing FIXED PHI batch {batch_idx}: {start_idx} to {end_idx}")
    print(f"{'='*80}")
    
    # Get pce_df subset for this batch
    pce_df_subset = pce_df_full[start_idx:end_idx].copy().reset_index(drop=True)
    
    # === FIXED PHI CHECKPOINTS ===
    fixed_ckpt_path = f'/Users/sarahurbut/Library/CloudStorage/Dropbox/enrollment_predictions_fixedphi_withpcs_fromclaudeoutput/model_enroll_fixedphi_sex_{start_idx}_{end_idx}.pt'
    
    try:
        fixed_ckpt = torch.load(fixed_ckpt_path, weights_only=False)
        model.load_state_dict(fixed_ckpt['model_state_dict'])
        
        # For fixed phi, Y is not saved in checkpoint, so extract from full Y tensor
        # Extract batch from full tensors
        Y_batch = Y_full[start_idx:end_idx]
        E_batch = E_full[start_idx:end_idx]
        
        # Update model.Y and model.N so forward() uses correct patients
        model.Y = torch.tensor(Y_batch, dtype=torch.float32)
        model.N = Y_batch.shape[0]  # Update N to match new Y size
       
        # 10-year predictions
        print(f"\nFixed Phi - 10 year predictions...")
        fixed_10yr = evaluate_major_diseases_wsex_with_bootstrap_dynamic(
            model, Y_batch, E_batch, disease_names, pce_df_subset, 
            n_bootstraps=100, follow_up_duration_years=10, patient_indices=None
        )
        fixed_10yr['batch_idx'] = batch_idx
        fixed_10yr_results.append(fixed_10yr)
        
        # 30-year predictions
        print(f"\nFixed Phi - 30 year predictions...")
        fixed_30yr = evaluate_major_diseases_wsex_with_bootstrap_dynamic(
            model, Y_batch, E_batch, disease_names, pce_df_subset, 
            n_bootstraps=100, follow_up_duration_years=30, patient_indices=None
        )
        fixed_30yr['batch_idx'] = batch_idx
        fixed_30yr_results.append(fixed_30yr)
        
        # Static 10-year predictions (using 1-year score)
        print(f"\nFixed Phi - Static 10 year predictions (1-year score)...")
        fixed_static_10yr = evaluate_major_diseases_wsex_with_bootstrap(
            model=model,
            Y_100k=Y_batch,
            E_100k=E_batch,
            disease_names=disease_names,
            pce_df=pce_df_subset,
            n_bootstraps=100,
            follow_up_duration_years=10,
        )
        fixed_static_10yr['batch_idx'] = batch_idx
        fixed_static_10yr_results.append(fixed_static_10yr)
        
    except FileNotFoundError:
        print(f"Fixed phi checkpoint not found: {fixed_ckpt_path}")
        continue
    except Exception as e:
        print(f"Error processing fixed phi checkpoint {batch_idx}: {e}")
        continue

print(f"\n{'='*80}")
print("Completed processing all FIXED PHI checkpoints!")
print(f"{'='*80}")


Loading full Y and E tensors...


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/2217820238.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Y_full = torch.load('/Users/sarahurbut/Librar

Loaded full tensors!

Processing FIXED PHI batch 0: 0 to 10000


/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/2217820238.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model.Y = torch.tensor(Y_batch, dtype=torch.float32)



Fixed Phi - 10 year predictions...

Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.608 (0.585-0.622) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 831 (8.3%) (from 10000 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.574 (0.553-0.594) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 581 (5.8%) (from 10000 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.599 (0.576-0.626) (calculated on 9864 individuals)
Events (10-Year in Eval Cohort): 376 (3.8%) (from 9864 individuals)
Excluded 136 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.713 (0.679-0.748) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 207 (2.1%) (from 10000 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.651 (0.632-0.668) (calculated on 10000 individuals)
Eve

...


Fixed Phi - 10 year predictions...

Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.627 (0.611-0.648) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 895 (8.9%) (from 10000 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.588 (0.567-0.611) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 575 (5.8%) (from 10000 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.605 (0.581-0.631) (calculated on 9856 individuals)
Events (10-Year in Eval Cohort): 385 (3.9%) (from 9856 individuals)
Excluded 144 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.687 (0.660-0.713) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 228 (2.3%) (from 10000 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.624 (0.603-0.644) (calculated on 10000 individuals)
Eve

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/2217820238.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model.Y = torch.tensor(Y_batch, dtype=torch.float32)



Fixed Phi - 10 year predictions...

Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.638 (0.626-0.654) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 883 (8.8%) (from 10000 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.577 (0.556-0.601) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 603 (6.0%) (from 10000 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.599 (0.572-0.624) (calculated on 9874 individuals)
Events (10-Year in Eval Cohort): 350 (3.5%) (from 9874 individuals)
Excluded 126 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.683 (0.651-0.717) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 221 (2.2%) (from 10000 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.639 (0.613-0.656) (calculated on 10000 individuals)
Eve

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_5551/2217820238.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  model.Y = torch.tensor(Y_batch, dtype=torch.float32)



Fixed Phi - 10 year predictions...

Evaluating ASCVD (Dynamic 10-Year Risk)...
AUC: 0.637 (0.622-0.656) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 853 (8.5%) (from 10000 individuals)
Excluded 0 prevalent cases for ASCVD.

Evaluating Diabetes (Dynamic 10-Year Risk)...
AUC: 0.603 (0.582-0.624) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 591 (5.9%) (from 10000 individuals)
Excluded 0 prevalent cases for Diabetes.

Evaluating Atrial_Fib (Dynamic 10-Year Risk)...
AUC: 0.588 (0.564-0.611) (calculated on 9885 individuals)
Events (10-Year in Eval Cohort): 368 (3.7%) (from 9885 individuals)
Excluded 115 prevalent cases for Atrial_Fib.

Evaluating CKD (Dynamic 10-Year Risk)...
AUC: 0.729 (0.695-0.763) (calculated on 10000 individuals)
Events (10-Year in Eval Cohort): 213 (2.1%) (from 10000 individuals)
Excluded 0 prevalent cases for CKD.

Evaluating All_Cancers (Dynamic 10-Year Risk)...
AUC: 0.643 (0.619-0.665) (calculated on 10000 individuals)
Eve

In [89]:
# Compute median AUC across all batches
def compute_median_aucs(results_list):
    """Extract AUC values and compute medians for each disease"""
    if not results_list:
        return pd.DataFrame()
    
    # Get all disease names from first result (excluding batch_idx)
    disease_names_list = [k for k in results_list[0].keys() if k != 'batch_idx']
    
    auc_data = {disease: [] for disease in disease_names_list}
    
    for result in results_list:
        for disease in disease_names_list:
            if disease in result and 'auc' in result[disease]:
                auc_val = result[disease]['auc']
                if not np.isnan(auc_val):
                    auc_data[disease].append(auc_val)
    
    # Compute medians
    median_aucs = {}
    for disease, aucs in auc_data.items():
        if aucs:
            median_aucs[disease] = np.median(aucs)
        else:
            median_aucs[disease] = np.nan
    
    # Create DataFrame
    df = pd.DataFrame([median_aucs]).T
    df.columns = ['median_auc']
    df = df.sort_values('median_auc', ascending=False)
    
    return df

# Compute median AUC DataFrames
print("Computing median AUCs...")
joint_10yr_median_df = compute_median_aucs(joint_10yr_results)
joint_30yr_median_df = compute_median_aucs(joint_30yr_results)
fixed_10yr_median_df = compute_median_aucs(fixed_10yr_results)
fixed_30yr_median_df = compute_median_aucs(fixed_30yr_results)
joint_static_10yr_median_df = compute_median_aucs(joint_static_10yr_results)
fixed_static_10yr_median_df = compute_median_aucs(fixed_static_10yr_results)

print("\n" + "="*80)
print("JOINT PHI - 10 YEAR PREDICTIONS - Median AUC")
print("="*80)
print(joint_10yr_median_df)

print("\n" + "="*80)
print("JOINT PHI - 30 YEAR PREDICTIONS - Median AUC")
print("="*80)
print(joint_30yr_median_df)

print("\n" + "="*80)
print("FIXED PHI - 10 YEAR PREDICTIONS - Median AUC")
print("="*80)
print(fixed_10yr_median_df)

print("\n" + "="*80)
print("FIXED PHI - 30 YEAR PREDICTIONS - Median AUC")
print("="*80)
print(fixed_30yr_median_df)

print("\n" + "="*80)
print("JOINT PHI - STATIC 10 YEAR PREDICTIONS (1-year score) - Median AUC")
print("="*80)
print(joint_static_10yr_median_df)

print("\n" + "="*80)
print("FIXED PHI - STATIC 10 YEAR PREDICTIONS (1-year score) - Median AUC")
print("="*80)
print(fixed_static_10yr_median_df)

# Save to CSV files
joint_10yr_median_df.to_csv('joint_phi_10yr_median_auc.csv')
joint_30yr_median_df.to_csv('joint_phi_30yr_median_auc.csv')
fixed_10yr_median_df.to_csv('fixed_phi_10yr_median_auc.csv')
fixed_30yr_median_df.to_csv('fixed_phi_30yr_median_auc.csv')
joint_static_10yr_median_df.to_csv('joint_phi_static_10yr_median_auc.csv')
fixed_static_10yr_median_df.to_csv('fixed_phi_static_10yr_median_auc.csv')

print("\nMedian AUC DataFrames saved to CSV files.")


Computing median AUCs...

JOINT PHI - 10 YEAR PREDICTIONS - Median AUC
                      median_auc
Parkinsons              0.724464
CKD                     0.708086
Heart_Failure           0.697602
Osteoporosis            0.690270
Bladder_Cancer          0.683268
Stroke                  0.681467
Prostate_Cancer         0.662691
Lung_Cancer             0.660430
COPD                    0.656907
ASCVD                   0.653996
All_Cancers             0.653568
Pneumonia               0.644770
Colorectal_Cancer       0.607017
Secondary_Cancer        0.605406
Diabetes                0.599016
Multiple_Sclerosis      0.594623
Thyroid_Disorders       0.588120
Anemia                  0.581611
Rheumatoid_Arthritis    0.573449
Crohns_Disease          0.542644
Ulcerative_Colitis      0.541622
Bipolar_Disorder        0.539409
Psoriasis               0.533948
Atrial_Fib              0.528289
Asthma                  0.521254
Anxiety                 0.514116
Breast_Cancer           0.513429
Depre

In [90]:
# Compute median AUC across all batches
def compute_mean_aucs(results_list):
    """Extract AUC values and compute medians for each disease"""
    if not results_list:
        return pd.DataFrame()
    
    # Get all disease names from first result (excluding batch_idx)
    disease_names_list = [k for k in results_list[0].keys() if k != 'batch_idx']
    
    auc_data = {disease: [] for disease in disease_names_list}
    
    for result in results_list:
        for disease in disease_names_list:
            if disease in result and 'auc' in result[disease]:
                auc_val = result[disease]['auc']
                if not np.isnan(auc_val):
                    auc_data[disease].append(auc_val)
    
    # Compute medians
    mean_aucs = {}
    for disease, aucs in auc_data.items():
        if aucs:
            mean_aucs[disease] = np.mean(aucs)
        else:
            mean_aucs[disease] = np.nan
    
    # Create DataFrame
    df = pd.DataFrame([mean_aucs]).T
    df.columns = ['mean_auc']
    df = df.sort_values('mean_auc', ascending=False)
    
    return df

# Compute median AUC DataFrames
print("Computing mean AUCs...")
joint_10yr_mean_df = compute_mean_aucs(joint_10yr_results)
joint_30yr_mean_df = compute_mean_aucs(joint_30yr_results)
fixed_10yr_mean_df = compute_mean_aucs(fixed_10yr_results)
fixed_30yr_mean_df = compute_mean_aucs(fixed_30yr_results)
joint_static_10yr_mean_df = compute_mean_aucs(joint_static_10yr_results)
fixed_static_10yr_mean_df = compute_mean_aucs(fixed_static_10yr_results)

print("\n" + "="*80)
print("JOINT PHI - 10 YEAR PREDICTIONS - Mean AUC")
print("="*80)
print(joint_10yr_mean_df)

print("\n" + "="*80)
print("JOINT PHI - 30 YEAR PREDICTIONS - Mean AUC")
print("="*80)
print(joint_30yr_mean_df)

print("\n" + "="*80)
print("FIXED PHI - 10 YEAR PREDICTIONS - Mean AUC")
print("="*80)
print(fixed_10yr_mean_df)

print("\n" + "="*80)
print("FIXED PHI - 30 YEAR PREDICTIONS - Mean AUC")
print("="*80)
print(fixed_30yr_mean_df)

print("\n" + "="*80)
print("JOINT PHI - STATIC 10 YEAR PREDICTIONS (1-year score) - Mean AUC")
print("="*80)
print(joint_static_10yr_mean_df)

print("\n" + "="*80)
print("FIXED PHI - STATIC 10 YEAR PREDICTIONS (1-year score) - Mean AUC")
print("="*80)
print(fixed_static_10yr_mean_df)

# Save to CSV files
joint_10yr_median_df.to_csv('joint_phi_10yr_mean_auc.csv')
joint_30yr_median_df.to_csv('joint_phi_30yr_mean_auc.csv')
fixed_10yr_median_df.to_csv('fixed_phi_10yr_mean_auc.csv')
fixed_30yr_median_df.to_csv('fixed_phi_30yr_mean_auc.csv')
joint_static_10yr_median_df.to_csv('joint_phi_static_10yr_mean_auc.csv')
fixed_static_10yr_median_df.to_csv('fixed_phi_static_10yr_mean_auc.csv')

print("\nMean AUC DataFrames saved to CSV files.")

Computing mean AUCs...

JOINT PHI - 10 YEAR PREDICTIONS - Mean AUC
                      mean_auc
Parkinsons            0.721116
CKD                   0.706205
Heart_Failure         0.699092
Osteoporosis          0.689301
Bladder_Cancer        0.685003
Stroke                0.682043
Lung_Cancer           0.664386
Prostate_Cancer       0.661342
COPD                  0.656095
All_Cancers           0.653230
ASCVD                 0.652882
Pneumonia             0.643167
Secondary_Cancer      0.605142
Colorectal_Cancer     0.602444
Multiple_Sclerosis    0.599871
Diabetes              0.598152
Thyroid_Disorders     0.586794
Anemia                0.580701
Rheumatoid_Arthritis  0.574710
Ulcerative_Colitis    0.542664
Crohns_Disease        0.540576
Bipolar_Disorder      0.533674
Atrial_Fib            0.529457
Psoriasis             0.526381
Asthma                0.523253
Breast_Cancer         0.515143
Anxiety               0.513754
Depression            0.485151

JOINT PHI - 30 YEAR PREDICTIONS -